## Setup

In [40]:
import os
import sys
import logging
import pandas as pd
from google.cloud import bigquery
from hashlib import md5
from typing import List


# **** SETUP ****
PROJECT_NAME = "deb-01-372112"
DATASET_NAME = "tickets"
DATA_DIR = "./data/"
DEFAULT_TICKETS_FILE = os.path.join(DATA_DIR, "tickets.json")

# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'airlines': {
        'table_name': 'airlines',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('iata', 'string', mode='REQUIRED'),
            bigquery.SchemaField('name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('icao', 'string', mode='REQUIRED'),
            bigquery.SchemaField('callsign', 'string', mode='REQUIRED'),
            bigquery.SchemaField('country', 'string', mode='REQUIRED'),
        ],
    },
    'airports': {
        'table_name': 'airports',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('iata', 'string', mode='REQUIRED'),
            bigquery.SchemaField('city', 'string', mode='REQUIRED'),
            bigquery.SchemaField('country', 'string', mode='NULLABLE'),
            bigquery.SchemaField('name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('icao', 'string', mode='REQUIRED'),
            bigquery.SchemaField('latitude', 'float', mode='REQUIRED'),
            bigquery.SchemaField('longitude', 'float', mode='REQUIRED'),
            bigquery.SchemaField('altitude', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('tz_timezone', 'string', mode='REQUIRED')
        ],
    },
  'passengers': {
        'table_name': 'passengers',
        'schema': [
            bigquery.SchemaField('passenger_sk', 'string', mode='REQUIRED'),
            bigquery.SchemaField('first_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('last_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('gender', 'string', mode='REQUIRED'),
            bigquery.SchemaField('birth_date', 'date_time', mode='REQUIRED'),
            bigquery.SchemaField('email', 'string', mode='REQUIRED'),
            bigquery.SchemaField('street', 'string', mode='REQUIRED'),
            bigquery.SchemaField('city', 'string', mode='REQUIRED'),
            bigquery.SchemaField('state', 'string', mode='REQUIRED'),
            bigquery.SchemaField('zip', 'string', mode='REQUIRED'),
            bigquery.SchemaField('start_date', 'date_time', mode='REQUIRED'),
            bigquery.SchemaField('end_date', 'date_time', mode='REQUIRED'),

        ]
    }  
}


# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")

[DEBUG][2023-01-13 11:08:09,334][1559942662:0078] : Creating bigquery client
[INFO ][2023-01-13 11:08:09,338][1559942662:0081] : Setup Completed


## Create Tickets' BQ Dataset

In [41]:
dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

logger.info(f"Created Tickets dataset: {dataset.full_dataset_id}")

[INFO ][2023-01-13 11:08:12,827][2665781056:0006] : Created Tickets dataset: deb-01-372112:tickets


## Load Original Tickets File

In [42]:
import json 
# receipts data file name
filename = DEFAULT_TICKETS_FILE
logger.debug(f"attempting to process: {filename}")

# *** always perform checks first ***
# check if the file exists
assert os.path.exists(filename), f"Data file does not exists: '{filename}'"
# check if the file contains any data. Header row alone is 78 bytes. size > 78
assert os.path.getsize(filename) > 78, f"Data file size incorrect; does not seem to contain data: '{filename}'"

# load into dataframe
with open('./data/tickets.json') as data_file:    
    data = [json.loads(line) for line in data_file]

df = pd.json_normalize(data)
logger.info(f"loaded {len(df.index)} rows from: {filename}")

# *** always perform check at the end ***
# check schema: contains all expected columns?
expected_columns = ['eticket_num', 'confirmation', 'ticket_date', 'price', 'seat', 'status', 'origin', 'destination']
for col in expected_columns:
    assert col in list(df.columns), f"Data file missing required column: {col}"

# assign & remember receipts dataframe
tickets_df = df
display(tickets_df.head(n=10))

[DEBUG][2023-01-13 11:08:14,506][2599063938:0004] : attempting to process: ./data/tickets.json
[INFO ][2023-01-13 11:08:14,963][2599063938:0017] : loaded 4096 rows from: ./data/tickets.json


,eticket_num,confirmation,ticket_date,price,seat,status,airline.name,airline.iata,airline.icao,airline.callsign,...,passenger.last_name,passenger.gender,passenger.birth_date,passenger.email,passenger.street,passenger.city,passenger.state,passenger.zip,origin,destination
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,China Eastern Airlines,MU,CES,CHINA EASTERN,...,Brown,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN
1,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,Hawaiian Airlines,HA,HAL,HAWAIIAN,...,Kent,F,1998-08-05,laura.kent.98@hotmail.com,13991 Davis Village,North Catherineborough,PA,16516,NaN,NaN
2,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,Wizz Air,W6,WZZ,WIZZ AIR,...,Tucker,F,1965-01-22,lisa.tucker.65@hotmail.com,04135 Marvin Via,North Kristabury,MA,01093,NaN,NaN
3,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,AirAsia,AK,AXM,ASIAN EXPRESS,...,Yates,NB,1975-03-31,matthew.yates.75@yahoo.com,76045 Samantha Road Suite 111,Lake Jeffrey,DE,19898,NaN,NaN
4,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,Xiamen Airlines,MF,CXA,XIAMEN AIR,...,Villanueva,NB,1945-08-14,megan.villanueva.45@hotmail.com,848 Melissa Springs Suite 947,Kellerstad,TX,76177,NaN,NaN
5,115-196069-8963,XFYQC0,2022-03-23,892.69,18C,active,Air New Zealand,NZ,ANZ,NEW ZEALAND,...,Hall,NB,1944-08-31,sarah.hall.44@gmail.com,75420 Michael Mountains Suite 485,New Victoria,HI,96727,NaN,NaN
6,396-673460-1326,N5UOOZ,2022-03-23,889.53,3C,active,Jeju Air,7C,JJA,JEJU AIR,...,Thompson,M,1968-05-02,seth.thompson.68@yahoo.com,22455 Higgins Junction Apt. 042,New Keith,OR,97405,NaN,NaN
7,380-894599-8109,PAA19Y,2022-03-22,706.78,7D,active,American Airlines,AA,AAL,AMERICAN,...,Garcia,F,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,NaN,NaN
8,614-960971-2686,EF4BHJ,2022-03-23,486.40,24J,active,Juneyao Airlines,HO,DKH,JUNEYAO AIRLINES,...,Clark,F,1991-11-09,becky.clark.91@gmail.com,691 Jones Cliffs,Michaelburgh,TX,76003,NaN,NaN
9,481-321233-0702,FVM9EE,2022-03-23,855.93,16A,active,Royal Air Maroc,AT,RAM,ROYALAIR MAROC,...,Cook,M,1976-07-29,ronald.cook.76@hotmail.com,93328 Davis Island,Rodriguezside,MD,21408,NaN,NaN


## Load Airlines

In [47]:
# start from the tickets
df = tickets_df

logger.debug(f"getting uniques products...")

# set of unique columns to return
cols = ['airline.name', 'airline.iata', 'airline.icao', 'airline.callsign', 'airline.country']

# group by unique columns and only select them
df = df.groupby(cols).all()
df = df.reset_index().loc[:, cols]

# rename columns
df = df.rename(columns={'airline.name': 'name', 'airline.iata': 'iata', 'airline.icao': 'icao', 'airline.callsign': 'callsign', 'airline.country': 'country'})
# set index by iata
df = df.set_index(keys='iata')

logger.info(f"airlines dim - found {len(df.index)} rows")
display(df)

[DEBUG][2023-01-13 11:10:36,128][137921925:0004] : getting uniques products...
[INFO ][2023-01-13 11:10:36,487][137921925:0018] : airlines dim - found 48 rows


,name,icao,callsign,country
iata,,,,
AC,Air Canada,ACA,AIR CANADA,Canada
CA,Air China,CCA,AIR CHINA,China
AF,Air France,AFR,AIRFRANS,France
NZ,Air New Zealand,ANZ,NEW ZEALAND,New Zealand
AK,AirAsia,AXM,ASIAN EXPRESS,Malaysia
AS,Alaska Airlines,ASA,Inc.,ALASKA
G4,Allegiant Air,AAY,ALLEGIANT,United States
AA,American Airlines,AAL,AMERICAN,United States
BA,British Airways,BAW,SPEEDBIRD,United Kingdom


## Loading to BigQuery

In [35]:
#function for loading table to BigQuery 
def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

In [48]:
# get table name and schema from our TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['airlines']['table_name']}"
schema = schema=TABLE_METADATA['airlines']['schema']
# load dataframe
load_table(df, client, table_name, schema)

logger.info(f"loaded products dim")

[INFO ][2023-01-13 11:10:44,718][1947479415:0029] : loading table: 'deb-01-372112.tickets.airlines'
[INFO ][2023-01-13 11:10:48,556][1947479415:0034] : loaded 48 rows into deb-01-372112:tickets.airlines
[INFO ][2023-01-13 11:10:48,561][2329088183:0007] : loaded products dim


## Load Airports

In [60]:
# start from the tickets
df = tickets_df

logger.debug(f"getting unique aiports...")

airports_cols = ['origin.name', 'origin.city', 'origin.country', 'origin.iata', 'origin.icao', 'origin.latitude', 'origin.longitude', 'origin.altitude', 'origin.tz_timezone']

# get unique airports and only selecy them
df = df.groupby(airports_cols).all()
df = df.reset_index().loc[:, airports_cols]

#rename columns
df = df.rename(columns={'origin.name': 'name', 'origin.city': 'city', 'origin.country': 'country', 'origin.iata': 'iata', 'origin.icao': 'icao', 'origin.latitude': 'latitude', 'origin.longitude': 'longitude', 'origin.altitude': 'altitude', 'origin.tz_timezone': 'tz_timezone'})
df = df.set_index(keys='iata')
logger.info(f"customers dim - found {len(df.index)} rows")

display(df)

[DEBUG][2023-01-13 11:31:20,678][2564274586:0004] : getting unique aiports...
[INFO ][2023-01-13 11:31:20,978][2564274586:0015] : customers dim - found 386 rows


,name,city,country,icao,latitude,longitude,altitude,tz_timezone
iata,,,,,,,,
AUH,Abu Dhabi International Airport,Abu Dhabi,United Arab Emirates,OMAA,24.43,54.65,88.0,Asia/Dubai
MAD,Adolfo Suárez Madrid–Barajas Airport,Madrid,Spain,LEMD,40.47,-3.56,1998.0,Europe/Madrid
CWB,Afonso Pena Airport,Curitiba,Brazil,SBCT,-25.53,-49.18,2988.0,America/Sao_Paulo
MCP,Alberto Alcolumbre Airport,Macapa,Brazil,SBMQ,0.05,-51.07,56.0,America/Fortaleza
ABQ,Albuquerque International Sunport,Albuquerque,United States,KABQ,35.04,-106.61,5355.0,America/Denver
...,...,...,...,...,...,...,...,...
ZUH,Zhuhai Jinwan Airport,Zhuhai,China,ZGSD,22.01,113.38,23.0,Asia/Shanghai
MCZ,Zumbi dos Palmares Airport,Maceio,Brazil,SBMO,-9.51,-35.79,387.0,America/Fortaleza
ZRH,Zürich Airport,Zurich,Switzerland,LSZH,47.46,8.55,1416.0,Europe/Zurich


## Loading to BigQuery

In [62]:
# get the table name and schema from TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['airports']['table_name']}"
schema = schema=TABLE_METADATA['airports']['schema']
# load the dataframe
load_table(df, client, table_name, schema)
logger.info(f"loaded airports dim")

[INFO ][2023-01-13 11:33:30,646][1947479415:0029] : loading table: 'deb-01-372112.tickets.airports'
[INFO ][2023-01-13 11:33:33,914][1947479415:0034] : loaded 386 rows into deb-01-372112:tickets.airports
[INFO ][2023-01-13 11:33:33,917][339070838:0006] : loaded airports dim
